In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
# thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
thispath = os.getcwd()
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fnames, gui_fpath
from mesostat.metric.metric import MetricCalculator
from mesostat.utils.hdf5_io import DataStorage

from lib.gallerosalas.data_fc_db_raw import DataFCDatabase
import lib.analysis.coactivity as coactivity
from lib.common.visualization import merge_image_sequence_movie

%load_ext autoreload
%autoreload 2

In [ ]:
params = {}
# params['root_path_data']  = gui_fpath("Path to data collection",  './')
# params['root_path_data'] = '/media/alyosha/Data/TE_data/yasirdata_raw/'
params['root_path_data'] = '/media/aleksejs/DataHDD/work/data/yasir/yasirdata_raw'
# params['root_path_data'] = '/home/alyosha/data/yasirdata_raw/'

In [ ]:
dataDB = DataFCDatabase(params)

In [ ]:
mc = MetricCalculator(serial=True, verbose=False)

In [ ]:
print('mice', dataDB.mice)
print('nSessions', len(dataDB.sessions))
print('datatypes', dataDB.get_data_types())
print('nChannel', dataDB.get_nchannels('mou_5'))

# Why high correlations

### 1.1 Get 2 regions sufficiently far apart

In [ ]:
fig, ax = plt.subplots()

dataDB.plot_area_clusters(fig, ax, {'A': [0], 'B': [25]})
plt.show()

### 1.2 Compute correlation coefficient between them

### 1.3 Scatter plot their activity, color by parameters, seek why correlated

Initial conclusion:
* There is clear correlation, which appears as a singular 2D gaussian
   - No a-priori indication of multiclass behaviour
   - AUD+RSP (mostly RSP) code for trialType in M9, variance almost orthogonal to coding axis.
   
[TODO] Analyse relationship to behaviour

In [ ]:
mousename = 'mou_9'

df = pd.DataFrame()
for trialType in ['Hit', 'CR']:
    for intervName in ['TEX', 'DEL', 'REW']:
        for session in dataDB.get_sessions(mousename):
            dataRSP = dataDB.get_neuro_data({'session': session}, datatype='bn_trial',
                                            intervName=intervName, trialType=trialType)[0]
        
            dataRP = np.mean(dataRSP, axis=1)
            dfThis = pd.DataFrame()
            dfThis['AUD'] = dataRP[:, 0]
            dfThis['RSP'] = dataRP[:, 25]
            dfThis['Session'] = session
            dfThis['trialType'] = trialType
            dfThis['intervName'] = intervName
            df = df.append(dfThis)

In [ ]:
import seaborn as sns
from mesostat.utils.pandas_helper import pd_query

In [ ]:
sns.jointplot(data=df.sample(1000), x="AUD", y="RSP", hue="intervName", kind="scatter")

In [ ]:
sns.set(font_scale=2)
sns.pairplot(data=df, x_vars=['AUD', 'RSP', "Session"], y_vars=['AUD', 'RSP'], hue='trialType',
             height=8.27, aspect=11.7/8.27)

In [ ]:
sns.set(font_scale=1)
for session in dataDB.get_sessions(mousename):
    dfThis = pd_query(df, {'Session': session})
    sns.pairplot(data=dfThis, x_vars=['AUD', 'RSP'], y_vars=['AUD', 'RSP'], hue="intervName",
                 height=4, aspect=1)
    
#     sns.pairplot(data=dfThis, x_vars=['AUD', 'RSP'], y_vars=['AUD', 'RSP'], hue="trialType",
#              height=4, aspect=1)